In [2]:
import pandas as pd

# Specify dtype for article_id column to read it as a string
#dtype_dict = {'article_id': str}
#inter_df = pd.read_csv('/content/drive/MyDrive/data/transactions_train.csv', dtype=dtype_dict)
inter_df = pd.read_csv('/content/drive/MyDrive/data/transactions_train.csv')

#inter_df = inter_df.sort_values(by='t_dat').set_index('t_dat')
#inter_df.index = pd.to_datetime(inter_df.index, format='%Y-%m-%d')
#latest_date = inter_df.index.max()
#start_of_week = latest_date - pd.Timedelta(days=7)
#prior_6_weeks = latest_date - pd.Timedelta(days=42)

inter_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [5]:
# WORKS ON 0.1% OF DATA using ~4 GB RAM and generating results for 30k customers
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.preprocessing import LabelEncoder

# Assuming inter_df is your full dataset

# Initialize the LabelEncoders
customer_encoder = LabelEncoder()
article_encoder = LabelEncoder()

# Sample 0.1% of the data for training
train_df_sampled = inter_df.sample(frac=0.001)

# Encoding user and item identifiers on the sampled data
customer_encoder.fit(train_df_sampled['customer_id'])
article_encoder.fit(train_df_sampled['article_id'])

# Fit and transform customer_id and article_id for the sampled data
encoded_customer_id_sampled = customer_encoder.transform(train_df_sampled['customer_id'])
encoded_article_id_sampled = article_encoder.transform(train_df_sampled['article_id'])

# Prepare the dataset with LightFM's format for the sampled data
dataset_sampled = Dataset()
dataset_sampled.fit(encoded_customer_id_sampled, encoded_article_id_sampled)

# Building the interaction matrix for the sampled data
(interactions_sampled, weights_sampled) = dataset_sampled.build_interactions(zip(encoded_customer_id_sampled, encoded_article_id_sampled))

# Fit the model on the sampled data
model = LightFM(loss='warp')
model.fit(interactions_sampled, sample_weight=weights_sampled, epochs=10)

# Define the batched_sample_recommendation function
def batched_sample_recommendation(model, dataset, customer_encoder, article_encoder, batch_size=1000):
    n_users, n_items = dataset.interactions_shape()
    all_predictions = []

    for start_index in range(0, n_users, batch_size):
        end_index = min(start_index + batch_size, n_users)
        batch_customer_idxs = np.arange(start_index, end_index)

        all_item_idxs = np.tile(np.arange(n_items), (len(batch_customer_idxs), 1))
        batch_customer_idxs_expanded = np.repeat(batch_customer_idxs[:, np.newaxis], n_items, axis=1)

        # Predict scores for the batch
        batch_scores = model.predict(batch_customer_idxs_expanded.flatten(), all_item_idxs.flatten()).reshape(len(batch_customer_idxs), n_items)

        # Get top 12 items for each user in the batch
        top_items_idxs = np.argpartition(-batch_scores, 11, axis=1)[:, :12]

        # Flatten, inverse transform, and reshape
        top_items_flat = article_encoder.inverse_transform(top_items_idxs.flatten())
        top_items_reshaped = top_items_flat.reshape(-1, 12)

        all_predictions.extend(top_items_reshaped)

    return all_predictions


# Define the create_predictions_df function
def create_predictions_df(customer_encoder, predictions):
    customer_ids = customer_encoder.classes_
    formatted_predictions = []

    for pred in predictions:
        # Format each article_id with leading zeros to make it 10 digits long
        formatted_pred = ["{:010d}".format(int(article)) for article in pred]
        formatted_predictions_str = ' '.join(formatted_pred)
        formatted_predictions.append(formatted_predictions_str)

    return pd.DataFrame({'customer_id': customer_ids, 'prediction': formatted_predictions})

# Use the batched_sample_recommendation function for prediction
top_items_predictions = batched_sample_recommendation(model, dataset_sampled, customer_encoder, article_encoder, batch_size=1000)

# Create the final DataFrame for all customers
predictions_df = create_predictions_df(customer_encoder, top_items_predictions)

# Now you can use predictions_df as needed
predictions_df.head()

,customer_id,prediction
0,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0316657001 0504154022 0580684001 0307239001 06...
1,000608ab13228c9d4f90f2e7e7dfd3b0c280d84ae817bf...,0664949002 0504154022 0571197007 0659460001 05...
2,00079f6287599e8e3f3558e6787e8a2a241e6536b15a7a...,0541308022 0300024063 0108775051 0599580072 06...
3,00080403a669b3b89d1bef1ec73ea466d95e39698d6dde...,0675139002 0456163028 0741342005 0579541035 05...
4,0008d30a148478dc88c69af6c51230ad5802590afc8488...,0581588009 0300024063 0316657001 0619884020 05...


In [7]:
# WORKS ON FULL DATA. 
# FOR KAGGLE SUBMISSION ALSO ADDED CUSTOMERS FROM customers.csv with most frequent prediction (10k customers didnt have transaction)
# Kaggle submission score Score: 0.00008  ;  Public score: 0.00023
    # jv_prediction_lightfm_1.csv
    # Submitted by Justs Viduss · Submitted 17 minutes ago
import pandas as pd

# Specify dtype for article_id column to read it as a string
#dtype_dict = {'article_id': str}
#inter_df = pd.read_csv('/content/drive/MyDrive/data/transactions_train.csv', dtype=dtype_dict)
inter_df = pd.read_csv('/content/drive/MyDrive/data/transactions_train.csv')

import numpy as np
#import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.preprocessing import LabelEncoder

# Assuming inter_df is your full dataset

# Initialize the LabelEncoders
customer_encoder = LabelEncoder()
article_encoder = LabelEncoder()

# Encode all customer_ids and article_ids
encoded_customer_ids = customer_encoder.fit_transform(inter_df['customer_id'])
encoded_article_ids = article_encoder.fit_transform(inter_df['article_id'])

# Initialize the LightFM dataset
dataset = Dataset()
dataset.fit(encoded_customer_ids, encoded_article_ids)

# Define the create_predictions_df function
def create_predictions_df(customer_encoder, predictions):
    customer_ids = customer_encoder.classes_
    formatted_predictions = []

    for pred in predictions:
        formatted_pred = ["{:010d}".format(int(article)) for article in pred]
        formatted_predictions_str = ' '.join(formatted_pred)
        formatted_predictions.append(formatted_predictions_str)

    return pd.DataFrame({'customer_id': customer_ids, 'prediction': formatted_predictions})

# Initialize the LightFM model
model = LightFM(loss='warp')

# Split data into chunks and train the model iteratively
chunk_size = int(0.001 * len(inter_df))  # 0.1% of the dataset size
for start in range(0, len(inter_df), chunk_size):
    end = min(start + chunk_size, len(inter_df))
    chunk = inter_df.iloc[start:end]

    # Transform customer_id and article_id for the chunk
    chunk_encoded_customer_id = customer_encoder.transform(chunk['customer_id'])
    chunk_encoded_article_id = article_encoder.transform(chunk['article_id'])

    # Building the interaction matrix for the chunk
    interactions, weights = dataset.build_interactions(zip(chunk_encoded_customer_id, chunk_encoded_article_id))

    # Fit the model on the chunk
    model.fit(interactions, sample_weight=weights, epochs=10)

# Use the model to make predictions for all customers
top_items_predictions = batched_sample_recommendation(model, dataset, customer_encoder, article_encoder, batch_size=1000)

# Create the final DataFrame for all customers
predictions_df = create_predictions_df(customer_encoder, top_items_predictions)

# Now you can use predictions_df as needed
predictions_df.head()


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0759814012 0902864003 0875950005 0891050001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0902864003 0902069001 0657497007 0911564001 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0902864003 0875950005 0904995002 0889816001 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0903518002 0902069001 0875950005 0926387001 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0903864001 0889816001 0911564001 0875950005 06...
